<hr style="border:2px solid gray"> </hr>

# <center>FakeQuakes Tutorial</center>
## <center>Generates synthetic rupture models and waveforms </center>
### <center>Tara Nye</center>
### <center>PhD Candidate | University of Oregon</center>

<hr style="border:2px solid gray"> </hr>

FakeQuakes is a set of semistochastic foward modeling codes from the github repository [MudPy](https://github.com/dmelgarm/MudPy).  FakeQuakes was created by __Dr. Diego Melgar__ (professor at the Unviersity of Oregon), and the components are outlined in the paper [__Kinematic rupture scenarios and synthetic displacement data: An example application to the Cascadia subduction zone__ (Melgar et al., 2016)](http://doi.wiley.com/10.1002/2016JB013314).  

This set of codes generates synthetic rupture models and can generate low frequency displacement waveforms and broadband acceleration waveforms.

For more information on how to use FakeQuakes, you can refer to this [wiki page](https://github.com/taranye96/tsuquakes/wiki/FakeQuakes-Script) on my github account.  This page walks you through how to set up the parameter file (__.fq.py__) listed in the [tsuquakes](https://github.com/taranye96/tsuquakes) respository and how to run the different steps in FakeQuakes.  If you have any other questions, feel free to contact me or Diego Melgar.

Tara Nye: tnye@uoregon.edu
<br>Diego Melgar: dmelgarm@uoregon.edu

<img src="./figures/rupture_models.png"/>

<br>__Figure 1.__ Mean slip model for the 2010 M7.8 Mentawai event on the left (modified from Yue et al., 2014), and two example synthetic rupture models patterned after this event in the center and on the right.

<hr style="border:1px solid gray"> </hr>

# <center>Project Setup</center>

## Initial questions to ask before generating data:
### 1. What type of earthquake (thrust, normal, strike-slip)?
### 2. What is my target magnitude (or magnitude range)?
### 3. Where is my source region?
### 4. What is the regional velocity structure?
### 5. What type of data (displacement, acceleration)?
### 6. What stations do I want to use? 

<a id='directories'></a>
***
## <center>Setup Directories</center>
### In the FakeQuakes_tutorial folder, there is a files folder with these types of files:
- __.fault__ (fault model with subfault dimensions that will be used to generate the rupture scenarios)
- __.mod__ (velocity profile model for the region where the synthetic scenarios are being generated)
- __.gflist__ (list of the stations and their coordinates)

### __.fault__ 
(Modified from Yue et al., 2014)

In [ ]:
!head ./files/mentawai.fault
# Subfault, lon, lat, depth(km), strike, dip, type(doesn't do anything), risetime(doesn't do anything), 
# Subfault length(m), subfault width(m)

# The columns that don't do anything are left over from slip inversion file setup, but need to be included 
# in the structure of the file. 

### __.mod__

In [ ]:
!head ./files/mentawai.mod
# Thickness of layer(km), Vs(km/s), Vp(km/s), Density(g/cm^3), QS, QP
# Last area has to be thickness of 0 (infinite below this)

### __.gflist__ 

In [ ]:
!head ./files/gnss.gflist

### We need to initialize the project folder and put these files into their respective folders

In [ ]:
# Standard Library Imports
import os
import numpy as np
from obspy.core import UTCDateTime
from obspy import read
import time

# MudPy Imports
from mudpy import fakequakes,runslip,forward,view

In [ ]:
# Define variables for project location 
home = os.getcwd() + '/' # This sets the current working directory as the home path
project_name='demo' # Name of project folder that will be set up in home directory 

If you don't have the dependencies installed, or you want to just use the data I created, set __project_name=demo__

In [ ]:
# Initalize project folders
if project_name != 'demo':
    fakequakes.init(home,project_name)

### Copy the files into these folders 
- __.fault__ should be placed in __*/home/project_name/data/model_info/*__
- __.mod__ should be placed in __*/home/project_name/structure/*__ 
- __.gflist__ file should be placed in __*/home/project_folder/data/station_info/*__

In [ ]:
!cp ./files/mentawai.fault $home/$project_name/data/model_info
!cp ./files/mentawai.mod $home/$project_name/structure
!cp ./files/gnss.gflist $home/$project_name/data/station_info

<a id='parameters'></a>

<a id='parameters'></a>
***
## <center>Define Parameters</center>
### Rupture parameters

In [ ]:
# Runtime parameters 
ncpus=2                                        # how many CPUS you want to use for parallelization (needs ot be at least 2)
Nrealizations=2                                # Number of fake ruptures to generate per magnitude bin
hot_start=0                                    # If code quits in the middle of running, it will pick back up at this index

# File parameters
model_name='mentawai.mod'                      # Velocity model file name
fault_name='mentawai.fault'                    # Fault model name
mean_slip_name=None                            # Set to path of .rupt file if patterning synthetic runs after a mean rupture model
run_name='mentawai'                            # Base name of each synthetic run (i.e. mentawai.000000, mentawai.000001, etc...)
rupture_list='ruptures.list'                   # Name of list of ruptures that are used to generate waveforms.  'ruptures.list' uses the full list of ruptures FakeQuakes creates. If you create file with a sublist of ruptures, use that file name.
distances_name='mentawai'                      # Name of matrix with estimated distances between subfaults i and j for every subfault pair
load_distances=0                               # This should be zero the first time you run FakeQuakes with your fault model.

In [ ]:
# Source parameters
UTM_zone='47N'                                 # UTM_zone for rupture region 
time_epi=UTCDateTime('2010-10-25T14:42:12Z')   # Origin time of event (can set to any time, as long as it's not in the future)
target_Mw=np.array([8.5])                      # Desired magnitude(s), can either be one value or an array
hypocenter=None                                # Coordinates of subfault closest to desired hypocenter, or set to None for random
force_hypocenter=False                         # Set to True if hypocenter specified
rake=90                                        # Average rake for subfaults
scaling_law='T'                                # Type of rupture: T for thrust, S for strike-slip, N for normal
force_magnitude=False                          # Set to True if you want the rupture magnitude to equal the exact target magnitude
force_area=False                               # Set to True if you want the ruptures to fill the whole fault model

In [ ]:
# Correlation function parameters
hurst=0.4                                      # Hurst exponent form Melgar and Hayes 2019
Ldip='auto'                                    # Correlation length scaling: 'auto' uses Melgar and Hayes 2019, 'MB2002' uses Mai and Beroza 2002
Lstrike='auto'                                 # Same as above
slip_standard_deviation=0.9                    # Standard deviation for slip statistics: Keep this at 0.9
lognormal=True                                 # Keep this as True to solve the problem of some negative slip subfaults that are produced

In [ ]:
# Rupture propagation parameters
rise_time_depths=[10,15]                       # Transition depths for rise time scaling (if slip shallower than first index, rise times are twice as long as calculated)
max_slip=40                                    # Maximum sip (m) allowed in the model
max_slip_rule=False                            # If true, uses a magntidude-depence for max slip
shear_wave_fraction=0.8                        # 0.8 is a standard value (Mai and Beroza 2002)
source_time_function='dreger'                  # options are 'triangle' or 'cosine' or 'dreger'
stf_falloff_rate=4                             # Only affects Dreger STF, 4-8 are reasonable values
num_modes=72                                   # Number of modes in K-L expansion
slab_name=None                                 # Slab 2.0 Ascii file for 3D geometry, set to None for simple 2D geometry
mesh_name=None                                 # GMSH output file for 3D geometry, set to None for simple 2D geometry

### Waveform Parameters

In [ ]:
# Green's Functions parameters
GF_list='gnss.gflist'                          # Stations file name
G_name='gnss'                                  # Basename you want for the Green's functions matrices
make_GFs=1                                     # This should be 1 to run Green's functions
make_synthetics=1                              # This should be 1 to make the synthetics
G_from_file=0                                  # This should be zero the first time you run FakeQuakes with your fault model and stations.

# fk parameters
# used to solve wave equation in frequency domain 
dk=0.1 ; pmin=0 ; pmax=1 ; kmax=20             # Should be set to 0.1, 0, 1, 20
custom_stf=None                                # Assumes specified source time function above if set to None

In [ ]:
# Low frequency waveform parameters
dt=0.5                                         # Sampling interval of LF data 
NFFT=512                                       # Number of samples in LF waveforms (should be in powers of 2)

# dt*NFFT = length of low-frequency dispalcement record

### Whew...we're finally done setting up the parameters

<a id='generate_ruptures'></a>

<a id='generate_ruptures'></a>
<hr style="border:1px solid gray"> </hr>

# <center>Generate Rupture Models</center>
This step uses the __fakequakes.generate_ruptures__ function to generate the different rupture models.  The outputs are .rupt files which are the rupture models with information on each subfault, and .log files which give details on the rupture, such as magntidue, hypocenter, origin time, etc...  Most of this is taken from Mai and Beroza (2002).  
<br>There are two main types of rupture models you can make:
- a rupture model patterned after a real or average rupture model.
- a rupture model with completely random slip patterns

<br> For this tutorial, we will be making random rupture models for the Mentawai region. 

In [ ]:
if project_name != 'demo':
    start = time.time()
   
    #Generate rupture models
    fakequakes.generate_ruptures(home,project_name,run_name,fault_name,slab_name,mesh_name,load_distances,
        distances_name,UTM_zone,target_Mw,model_name,hurst,Ldip,Lstrike,num_modes,Nrealizations,rake,
        rise_time_depths,time_epi,max_slip,source_time_function,lognormal,slip_standard_deviation,scaling_law,
        ncpus,mean_slip_name=mean_slip_name,force_magnitude=force_magnitude,force_area=force_area,
        hypocenter=hypocenter,force_hypocenter=force_hypocenter,shear_wave_fraction=shear_wave_fraction,
        max_slip_rule=max_slip_rule)    
    
    end = time.time()
    time_elaps = end - start
    if time_elaps < 60:
        print(f'Duration: {round(time_elaps)} seconds')
    else:
        print(f'Duration: {round(time_elaps/60)} minutes')

### Rupture Dimensions
<img src="./figures/fault.png" style="width:400px;height:182px"/>
The dimensions of the fault rupture are determined by the target magnitude and rupture dimension scaling laws $\eqref{eq:scaling law length}$$\eqref{eq:scaling law width}$ (Blaser et al., 2010).

\begin{equation*}
log_{10}L = -2.37 + 0.57M_{w}
\label{eq:scaling law length} \tag{1}
\end{equation*}

\begin{equation*}
log_{10}W = -1.86 + 0.46M_{w}
\label{eq:scaling law width} \tag{2}
\end{equation*}

To add some variability in the rupture dimensions of events of the same size, FakeQuakes uses a stochastic approach to the scaling laws $\eqref{eq:stochastic scaling law}$.

\begin{equation*}
log_{10}L \sim N(a_{1} + b_{1}M_{w}, s^{2}_{xy})
\label{eq:stochastic scaling law} \tag{3}
\end{equation*}

### Slip Statistics
<img src="./figures/slip.png" style="width:400px;height:182px"/>

Unless a hypocenter is specified, a random subfault is chosen at the hypocenter.  
<br>The slip on each subfault is normally distributed, and the vector __s__ contains all of the subfault slips $\eqref{eq:slip vector}$.

\begin{equation*}
s \sim N(\mu, \hat{C})
\label{eq:slip vector} \tag{4}
\end{equation*}

<img src="./figures/slip_pdf.png" style="width:440px;height:350px"/>

To produce a synthetic event with the desired magnitude, the mean vector, $\mu$, has uniform slip and enough scalar moment to match the target magnitude we specified $\eqref{eq:moment}$.

\begin{equation*}
M_{0} = \rho LW\mu
\label{eq:moment} \tag{5}
\end{equation*}

$\hat{C}$ is the covariance matrix of the slip distribution and is a function of the standard deviation of slip at each subfault and the correlation between each ith and jth subfault pair.  FakeQuakes uses the __Von Karman correlation function__ $\eqref{eq:correlation function}$ because it represents fault slip patterns well. 

\begin{equation*}
C_{ij}(r_{ij}) = \frac{G_H(r_{ij})}{G_0(r_{ij})}
\label{eq:correlation function} \tag{6}
\end{equation*}

\begin{equation*}
G_{H} = r^{H}_{ij}K_{H}(r_{ij})
\label{eq:GH} \tag{7}
\end{equation*}

- r$_{ij}$ is a length measure between subfaults i and j<br>(__Note__: This is not the actual distance.  It is a value dependent on the along strike and along dip distances between subfaults)
- __H__ is the hurst exponent, which describes the steepness of decay of the correlation function in the wavenumber domain (defined in parameters)
- K$_{H}$ is the modified Bessel function of the second kind
    - solution to Bessel differential equation which is used when solving the 3D wave equation

If $\lambda$$_{k}$ and v$_{k}$ are the eigenvalues and eigen vectors of the correlation function C$_{ij}$, then the slip vector, s, can be expressed by the __Karhunen Loeve (K-L) expansion__ $\eqref{eq:K-L expansion}$.  

\begin{equation*}
s = \mu + \sum\limits_{k=1}^N z_{k}\sqrt{\lambda_{k}v_{k}}
\label{eq:K-L expansion} \tag{8}
\end{equation*}

- N is the number of modes specified, and it can be any number up to the number of subaults in your mode.  The more modes you have, the better you can model the high frequency stuff, but it also takes longer to compute. 
- z$_{k}$ are normally distrubuted random numbers between 0-1 that are used as weights for each eigenmode


### Kinematics 
<img src="./figures/rupture.gif" style="width:400px;height:182px"/>
Now that we have the static slip distribution, we need to determine the kinematics of the rupture so that we can generate the waveforms.  

The rupture onset time of each subfault is determined __(2-stage)__
1. Background rupture distribution $\eqref{eq:Vrupt}$ is determined using the shear-wave speed from the velocity model (V$_{s}$) and the shear-wave fraction parameter we set (SF)
\begin{equation*}
V_{r} = (SF)(V_{s})
\label{eq:Vrupt} \tag{9}
\end{equation*}
    - There is a reduction in rupture speed at depths <10 km.  This is to account for the low shear-wave velocity at shallow depths in the accretionary wedge.
    - Subfault onset times from this background distribution are perturbed so that there's faster propagation where slip is large (Graves and Pitarka, 2010).
    - Risetime of each subfault (duration of slip) scaled by __√__(total slip on subfault) so that larger slips have longer risetimes
2. A slip rate function is applied to determine how the slip evolved over time. 

### View .log file

In [ ]:
logpath=f'{home}/{project_name}/output/ruptures/mentawai.000000.log'
var=!cat $logpath
var

### View .rupt file

In [ ]:
ruptpath=f'{home}{project_name}/output/ruptures/mentawai.000000.rupt'
!head $ruptpath

### Quick view of rupture models

In [ ]:
%matplotlib inline
#%matplotlib notebook
view.quick_model(f'{home}{project_name}/output/ruptures/mentawai.000000.rupt')
view.quick_model(f'{home}{project_name}/output/ruptures/mentawai.000001.rupt')

<hr style="border:1px solid gray"> </hr>

# <center>Generate Low Frequency Displacement Waveforms</center>

<a id='GFs'></a>
***
## <center>Compute Green's Functions</center>
This step uses the __runslip.inversion__ function to generate the Green's Functions, which are seismograms for each subfault-station pair...one step closer to getting the final waveforms!  This uses the frequency-waveform (fk) integration code from Zhu and Rivera (2002).

In [ ]:
# Increase number of CPUS to speed up computation time
ncpus=4

if project_name != 'demo':
    start = time.time()     
    
    # Comput Green's functions
    runslip.inversionGFs(home,project_name,GF_list,None,fault_name,model_name,
            dt,None,NFFT,None,make_GFs,make_synthetics,dk,pmin,
            pmax,kmax,0,time_epi,hot_start,ncpus,custom_stf,impulse=True) 
   
    end = time.time()
    time_elaps = end - start
    if time_elaps < 60:
        print(f'Duration: {round(time_elaps)} seconds')
    else:
        print(f'Duration: {round(time_elaps/60)} minutes')

### Green's Functions (GFs)
Green's functions are what the seismogram would look like for 1 meter of displacement, assuming a delta source.   
<br> To calculate the GFs, we assume a layered half-space model and use a propagator matrix, which is what we use to go from stress changes along the fault to displacement at the surface.
<table><tr>
<td> <img src="./figures/delta_source.png" style="width: 300px;"/> </td>
<td> <img src="./figures/half_space.png" style="width: 300px;"/> </td>
<td> <img src="./figures/propagator_matrix.png" style="width: 400px;" /> </td>
</tr></table>

<br>__Figure 2.__ (left) Delta source (*from thoughtco.com*), (middle) Half-space model (*from Zhu and Rivera, 2002*), and (right) Propagator matrix (*from Zhu and rivera, 2002*).

These Green's functions are made for every subfault-station pair.

<img src="./figures/subfault_seismograms.png" style="width:400px;height:250px"/>

<br>__Figure 3.__ Example subfault seismograms for one station (blue triangle).


### Synthetics
The GFs provide the static displacement, but we need to know how displcement evolved over time.  To obtain this, we convolve the GFs (*G*(*t*)) with a STF (*X*(*t*)) $\eqref{eq:synthetic}$.  FakeQuakes refers to these products as synthetics (*S*(*t*)).
 
\begin{equation*}
S(t) = X(t)\circledast G(t)
\label{eq:synthetic} \tag{9}
\end{equation*}

***
## <center>Make Low Frequency Waveforms</center>

We can use the __forwaqrd.waveforms_fakequakes__ function to get the displacement waveforms at the different stations in our stations file using the synthetics we obtained in the last step.  

In [ ]:
if project_name != 'demo':
    start = time.time()
    
    #Make low frequency waveforms
    forward.waveforms_fakequakes(home,project_name,fault_name,rupture_list,GF_list,
            model_name,run_name,dt,NFFT,G_from_file,G_name,source_time_function,
            stf_falloff_rate,hot_start=hot_start)
    
    end = time.time()
    time_elaps = end - start
    if time_elaps < 60:
        print(f'Duration: {round(time_elaps)} seconds')
    else:
        print(f'Duration: {round(time_elaps/60)} minutes')

The generation of LF waveforms is deterministic, which means it is based off of our understanding of how seismic waves propagate through a media.  This is done using a 1D velocity model and GFs.

The GFs assume slip of 1 m, so we have to scale them to match the actual slip on the fault by multiplying each synthetic by the slip on that subfault.  The final waveforms for each station are the sum of the subfault synthetics associated with the station.

### Read in the displacement waveforms

In [ ]:
st1 = read(f'{home}{project_name}/output/waveforms/mentawai.000000/BSAT.LYE.sac')
tr1 = st1[0]
st2 = read(f'{home}{project_name}/output/waveforms/mentawai.000000/BTHL.LYE.sac')
tr2 = st2[0]

tr1.plot()
tr2.plot()

<hr style="border:1px solid gray"> </hr>

# <center>Individual exercises</center>

***
## 1. Make your own .fault file

FakeQuakes has a function (__forward.makefault__) to create a .fault file given a set of parameters. 

Define parameters

In [ ]:
fout =                               # Path to save .fault file including the .fault extension name
strike =                             # Strike of subfaults
dip =                                # Dip of subfaults
rise_time =                          # Risetime of subfaults (this value doesn't matter for the .fault file because it gets computed when the rupture files are generated)
epicenter =                          # Center point of the fault [lon, lat, depth] (doesn't have to be the epicenter of your event)
nstrike =                            # Number of subfaults along strike
dx_dip =                             # Subfault length in the along-strike direction (km)
dx_strike =                          # Subfault length in the along-dip direction (km)
num_updip =                          # Number of rows of subfaults above hypocenter row
num_downdip =                        # Number of rows of subfaults below hypocenter row

Make fault file

In [ ]:
forward.makefault(fout,strike,dip,nstrike,dx_dip,dx_strike,epicenter,num_updip,num_downdip,rise_time)

View fault file

In [ ]:
from numpy import genfromtxt
import matplotlib.pyplot as plt

# Read in .fault file
f=genfromtxt(fout)

# Longitude and latitude of each subfault
lon=f[0:len(f),1]
lat=f[0:len(f),2]
    
# Plot subfaults
plt.figure(figsize=(5.2,10))
plt.scatter(lon,lat,marker='o')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.grid()
plt.show()

***
## 2. Modify velocity model

Examine how changes in the velocity model affect output waveforms.  You can adjust the thickness, velocity, or attenuation of the layers (or all of the above).

Modify the .mod file, and then run the code below to remake the rupture models and waveforms. You'll want to create a new project folder for this exercise so that you can compare the results with those using the original .mod file.  

In [ ]:
home = os.getcwd() + '/' # You can leave the home variable the same
project_name='' 

fakequakes.init(home,project_name)

Copy the input files into their respective folders

In [ ]:
!cp ./files/mentawai.fault $home/$project_name/data/model_info
!cp ./files/mentawai.mod $home/$project_name/structure
!cp ./files/gnss.gflist $home/$project_name/data/station_info

In [ ]:
# Generate rupture models
fakequakes.generate_ruptures(home,project_name,run_name,fault_name,slab_name,mesh_name,load_distances,
        distances_name,UTM_zone,target_Mw,model_name,hurst,Ldip,Lstrike,num_modes,Nrealizations,rake,
        rise_time_depths,time_epi,max_slip,source_time_function,lognormal,slip_standard_deviation,scaling_law,
        ncpus,mean_slip_name=mean_slip_name,force_magnitude=force_magnitude,force_area=force_area,
        hypocenter=hypocenter,force_hypocenter=force_hypocenter,shear_wave_fraction=shear_wave_fraction,
        max_slip_rule=max_slip_rule)    

# Compute Green's functions
runslip.inversionGFs(home,project_name,GF_list,None,fault_name,model_name,
            dt,None,NFFT,None,make_GFs,make_synthetics,dk,pmin,
            pmax,kmax,0,time_epi,hot_start,ncpus,custom_stf,impulse=True) 

# Make waveforms
forward.waveforms_fakequakes(home,project_name,fault_name,rupture_list,GF_list,
            model_name,run_name,dt,NFFT,G_from_file,G_name,source_time_function,
            stf_falloff_rate,hot_start=hot_start)

View the new waveforms

In [ ]:
st1 = read(f'{home}/{project_name}/output/waveforms/{run_name}.000000/BSAT.LYE.sac')
tr1 = st1[0]
st2 = read(f'{home}/{project_name}/output/waveforms/{run_name}.000000/BTHL.LYE.sac')
tr2 = st2[0]

tr1.plot()
tr2.plot()

***
## 3. Modify different rupture parameters

Try generating your own rupture scenarios by modifying the magnitude and hypocenter. You can also set __magnitude__ to be an array of several magnitudes to compare rupture models of different sizes.

In [ ]:
home = os.getcwd() + '/' # You can leave the home variable the same
project_name='' 

fakequakes.init(home,project_name)

Copy input files into respective folders

In [ ]:
!cp ./files/mentawai.fault $home/$project_name/data/model_info
!cp ./files/mentawai.mod $home/$project_name/structure
!cp ./files/gnss.gflist $home/$project_name/data/station_info

Define parameters

In [ ]:
magnitude = np.array([])
hypocenter = []

In [ ]:
# Generate rupture models
fakequakes.generate_ruptures(home,project_name,run_name,fault_name,slab_name,mesh_name,load_distances,
        distances_name,UTM_zone,target_Mw,model_name,hurst,Ldip,Lstrike,num_modes,Nrealizations,rake,
        rise_time_depths,time_epi,max_slip,source_time_function,lognormal,slip_standard_deviation,scaling_law,
        ncpus,mean_slip_name=mean_slip_name,force_magnitude=force_magnitude,force_area=force_area,
        hypocenter=hypocenter,force_hypocenter=force_hypocenter,shear_wave_fraction=shear_wave_fraction,
        max_slip_rule=max_slip_rule)    

View rupture models

In [ ]:
%matplotlib inline
#%matplotlib notebook
view.quick_model(f'{home}/{project_name}/output/{run_name}.000000.rupt')

***
## 4. Modify number of modes in the K-L expansion

The number of modes can be any number up to the number of subfaults you have.  The higher number of modes in the K-L expansion, the greater the fine-scale resolution in rupture model.  However, computation time is longer.  You can modify __num_modes__ to see how it affects the rupture models.  

In [ ]:
home = os.getcwd() + '/' # This sets the current working directory as the home path
project_name='' 

fakequakes.init(home,project_name)

Copy input files into respective folders

In [ ]:
!cp ./files/mentawai.fault $home/$project_name/data/model_info
!cp ./files/mentawai.mod $home/$project_name/structure
!cp ./files/gnss.gflist $home/$project_name/data/station_info

Define parameters

In [ ]:
num_modes = 

In [ ]:
# Generate rupture models
fakequakes.generate_ruptures(home,project_name,run_name,fault_name,slab_name,mesh_name,load_distances,
        distances_name,UTM_zone,target_Mw,model_name,hurst,Ldip,Lstrike,num_modes,Nrealizations,rake,
        rise_time_depths,time_epi,max_slip,source_time_function,lognormal,slip_standard_deviation,scaling_law,
        ncpus,mean_slip_name=mean_slip_name,force_magnitude=force_magnitude,force_area=force_area,
        hypocenter=hypocenter,force_hypocenter=force_hypocenter,shear_wave_fraction=shear_wave_fraction,
        max_slip_rule=max_slip_rule) 

View rupture models

In [ ]:
%matplotlib inline
#%matplotlib notebook
view.quick_model(f'{home}/{project_name}/output/{run_name}.000000.rupt')

<a id='BB'></a>
<hr style="border:1px solid gray"> </hr>

# <center> Final Notes </center>
FakeQuakes Products
* All rupture files are stored in __/output/ruptures/__
* All waveforms are stored in __/output/waveforms/__
    * __.LY*.sac__ files are the low-frequency displacement files

If you have already generated data for a certain fault file and set of stations, you can use the subfault distances and Green's functions matrices that were created previously, rather than make them from scratch every time.  This will speed up computation time for future runs. 
- set **G_from_file=1**
- set **load_distances=1**
- copy over .mseed files from **/GFs/matrices** from your previous project folder to your new one
- copy over files from **/data/distances** from your previous project folder to your new one

**Note:** If you ever change your .fault or .gflist files, you need to set **G_from_file** and **load_distances** = 0. 

<a id='references'></a>
<hr style="border:1px solid gray"> </hr>

### References
Blaser, L., F. Krüger, M. Ohrnberger, and F. Scherbaum (2010), Scaling relations of earthquake source parameter estimates with special focus on subduction environment, Bull. Seism. Soc. Am., 100, 2914–2926.

Graves, R., and A. Pitarka (2015), Refinements to the Graves and Pitarka (2010) broadband ground-motion simulation method, Seismol. Res. Lett., 86(1), 75–80.

Graves, R. W., & Pitarka, A. (2010), Broadband Ground-Motion Simulation Using a Hybrid Approach. Bulletin of the Seismological Society of America, 100(5A), 2095-2123. doi:10.1785/0120100057

Mai, P. M., and Beroza, G. C. (2002), A spatial random field model to characterize complexity in earthquake slip, J. Geophys. Res., 107(B11), 2308, doi:10.1029/2001JB000588

Melgar, D., Leveque, R. J., Dreger, D. S., & Allen, R. M. (2016), Kinematic rupture scenarios and synthetic displacement data: An example application to the Cascadia subduction zone. Journal of Geophysical Research: Solid Earth, 121(9), 6658-6674, doi:10.1002/2016jb013314

Yue, H., Lay, T., Rivera, L., Bai, Y., Yamazaki, Y., Cheung, K. F., … Muhari, A. (2014), Rupture process of the 2010 Mw 7.8 Mentawai tsunami earthquake from joint inversion of near-field hr-GPS and teleseismic body wave recordings constrained by tsunami observations. AGU: Journal of Geophysical Research, Solid Earth, 119, 5574–5593, doi:10.1002/2014JB011082

Zhu, L., & Rivera, L. A. (2002), A note on the dynamic and static displacements from a point source in multilayered media. Geophysical Journal International, 148(3), 619-627, doi:10.1046/j.1365-246x.2002.01610.x